# Chatbot for AI-Related Queries

## Install Libraries

Install dependancies

In [3]:
! pip install langchain_community \
llama-cpp-python \
datasets \
pinecone-client \
huggingface_hub \
gpt4all \
langchain-pinecone \
streamlit \
sentence-transformers \
langchain_huggingface \
langchain-pinecone \
nltk \
spacy \
transformers \
pypdf \
pyMuPDF \
pymupdf4llm \
langchain_google_genai

^C


## Import Libraries

Import Libraries

In [1]:
import os
import re
import time
import spacy
import logging
import pandas as pd


from datasets import load_dataset
from pinecone import Pinecone, ServerlessSpec
from tqdm import tqdm
from langchain_huggingface.llms import HuggingFaceEndpoint
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_pinecone.vectorstores import PineconeVectorStore
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFDirectoryLoader
from transformers import AutoTokenizer
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

d:\BroCamp\Projects\Chatbot-ai\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

API

In [ ]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACEHUB_API_TOKEN')
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ['PINECONE_API'] = os.getenv('PINECONE_API')

**Steps**

* Data Loading
* Data cleaning and intergration
* Create vector database
* Create embedding model
* Vectorize data
* Insert data into vector db
* Create UI
* Test

## **Load Data**

**Load arxiv paper dataset from huggingface**

In [ ]:
dataset_name = "jamescalam/llama-2-arxiv-papers-chunked"
data = load_dataset(path=dataset_name, split="train")
data

Convert data into pandas dataframe

In [ ]:
documents = data.to_pandas()
documents.head(2)

In [ ]:
documents.shape

(4838, 15)

**Load AI modern approach book pdf**

In [2]:
pdf_loader = PyPDFDirectoryLoader("../Data/")
pdf_documents = pdf_loader.load()

Impossible to decode XFormObject /Im10: Invalid Elementary Object starting with b'$' @348034: b' 3>>\nID x\x9c\x9d\x92\x8bn\x830\x0cEI $\xbc[\xfa\xff\x9fX\x1e\x01J\x02\x81\xec\xb2hl \xa6MuU\xc9J.\xc7\xd7\x8e\x89\xb5\xd6\xfb\x8aeYd\xd7\xce\xf3\x8c\x9c\x10\xc2C\xfc8\xfe\xc8\x9d\x00bc\xcc4i=i$\x9f2\x9a'


In [3]:
len(pdf_documents)

1166

In [4]:
pdf_documents[100]

Document(metadata={'producer': 'MiKTeX pdfTeX-1.40.20', 'creator': 'TeX', 'creationdate': '2021-03-27T18:10:43+05:30', 'author': 'Stuart Russell / Peter Norvig', 'ebx_publisher': 'Pearson Education, Limited', 'moddate': '2023-02-10T15:33:02+04:00', 'title': 'Artificial Intelligence: A Modern Approach, Global Edition, 4ed', 'source': '..\\Data\\ai_modern_approach_4_edi.pdf', 'total_pages': 1166, 'page': 100, 'page_label': '101'}, page_content='Section 3.4 Uninformed Search Strategies 101\nfunction BIBF-S EARCH (problemF , fF , problemB, fB) returns a solution node, or failure\nnodeF ←NODE (problemF .INITIAL ) // Node for a start state\nnodeB ←NODE (problemB.INITIAL ) // Node for a goal state\nfrontierF ←a priority queue ordered by fF , with nodeF as an element\nfrontierB ←a priority queue ordered by fB, with nodeB as an element\nreachedF ←a lookup table, with one key nodeF .STATE and value nodeF\nreachedB ←a lookup table, with one key nodeB.STATE and value nodeB\nsolution←failure\nwhile

In [5]:
book_df = pd.DataFrame([{'source':doc.metadata['source'],'page_label':doc.metadata['page_label'],'text':doc.page_content}for doc in pdf_documents[19:1073]])

In [6]:
book_df

,source,page_label,text
0,..\Data\ai_modern_approach_4_edi.pdf,20,20 Chapter 1 Introduction\nfour. The methods u...
1,..\Data\ai_modern_approach_4_edi.pdf,21,Section 1.1 What Is AI? 21\nproblems correctly...
2,..\Data\ai_modern_approach_4_edi.pdf,22,"22 Chapter 1 Introduction\nchange, and create ..."
3,..\Data\ai_modern_approach_4_edi.pdf,23,Section 1.2 The Foundations of Artiﬁcial Intel...
4,..\Data\ai_modern_approach_4_edi.pdf,24,24 Chapter 1 Introduction\n1.2.1 Philosophy\n•...
...,...,...,...
1049,..\Data\ai_modern_approach_4_edi.pdf,1069,Section 29.2 AI Architectures 1069\nﬁeld progr...
1050,..\Data\ai_modern_approach_4_edi.pdf,1070,1070 Chapter 29 The Future of AI\nAgents also ...
1051,..\Data\ai_modern_approach_4_edi.pdf,1071,Section 29.2 AI Architectures 1071\nThe truly ...
1052,..\Data\ai_modern_approach_4_edi.pdf,1072,1072 Chapter 29 The Future of AI\nA human bein...


In [ ]:
len(documents.iloc[0]['chunk'])

In [ ]:
len(book_df.iloc[0]['text'])

In [ ]:
book_df.iloc[0]['text']

## **Clean Data**

Clean Arvix Data
*  Removes all characters except letters (both uppercase and lowercase), digits, and whitespace.
* Replaces control characters (ASCII codes 0–31 and 127) with a space.
* Collapses multiple whitespace characters into a single space.
* Removes any extra whitespace that appears directly before punctuation marks.
* Ensures that there is a space after punctuation if one isn’t already present.
*  Replaces any left curly brace { with a hyphen -.
* Removes any right curly brace } from the text.
* Strips leading and trailing whitespace from each text string.

In [ ]:
def preprocess_doc(X):
    chunk1 = X.apply(lambda x: re.sub(r'[^a-zA-Z0-9.\s]', '', x))
    chunk2 = chunk1.apply(lambda x: re.sub(r'[\x00-\x1F\x7F]', ' ', x))
    chunk3 = chunk2.apply(lambda x: re.sub(r'\s+', ' ', x))
    chunk4 = chunk3.apply(lambda x: re.sub(r'\s+([,.!?;:])', r'\1', x))
    chunk5 = chunk4.apply(lambda x: re.sub(r'([,.!?;:])(?=\S)', r'\1 ', x))
    chunk6 = chunk5.apply(lambda x: re.sub(r'\{', '-', x))
    chunk7 = chunk6.apply(lambda x: re.sub(r'\}', '', x))
    chunk7 = chunk7.apply(lambda x: x.strip())
    return chunk7

**Clean PDF Data**
* Replace any character that is not an uppercase/lowercase letter, digit, or period with a space.
* Remove chapter and its number
* Remove section and its number


In [ ]:
def clean_book_data(X):
  X = re.sub(r'[^A-Za-z0-9.]',' ',X)
  pattern_chapter = r'^\d+\s+Chapter\s+\d+\s+.*?'
  re.sub(pattern_chapter, '', book_df.loc[100,'text'], count=1)
  X = re.sub(pattern_chapter, '', X, count=1)
  pattern_section = r'^\s+Section\s+\d+\s+.*?'
  pattern_section = r'^Section\s+\d+(?:\.\d+)?\s+.*?'
  X = re.sub(pattern_section, '', X, count=1)
  return X

In [ ]:
book_df

In [ ]:
book_df['text'] = book_df['text'].apply(clean_book_data)

In [ ]:
book_df

## **Dynamic Chunking**


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Function to dynamically chunk the text based on token size

In [ ]:
def dynamic_chunking(text,max_token=512,overlap=50):
  doc = nlp(text)
  chunks = []
  current_chunk = []
  token_length = 0
  for sent in doc.sents:
    token_length = len(tokenizer.tokenize(sent.text))
    if token_length + token_length <= max_token:
      current_chunk.append(sent.text)
      max_token += token_length
    else:
      if chunks:
        prev_tokens = tokenizer.tokenize(chunks[-1])
        overlap_tokens = prev_tokens[-overlap:]
        overlap_text = ' '.join(overlap_tokens)
        chunks.append(' '.join(current_chunk) + ' ' + overlap_text)
      else:
        chunks.append(' '.join(current_chunk))
      current_chunk = [sent.text]
      token_length = token_length
  if current_chunk:
    chunks.append(' '.join(current_chunk))
  return chunks


In [ ]:
book_df.head(3)

Apply above dynamic chunking into this book data

In [ ]:
dynamic_chunked_data = []
for _,row in tqdm(book_df.iterrows(), desc="Dynamically chunking data"):
  chunks = dynamic_chunking(row['text'])
  new_row = row.copy()
  for idx,chunk in enumerate(chunks):
    new_row['id'] = new_row['page_label'] + '-' + str(idx)
    new_row['text'] = chunk
    dynamic_chunked_data.append(new_row)

Dynamically chunking data: 1054it [01:34, 11.11it/s]


In [ ]:
dynamical_chunked_book_df = pd.DataFrame(dynamic_chunked_data)

In [ ]:
dynamical_chunked_book_df

Dynamic chunking Arvix Data

In [ ]:
dynamic_chunked_arvix = []
for _,row in tqdm(documents.iterrows(), desc="Dynamically chunking data"):
  chunks = dynamic_chunking(row['chunk'])
  new_row = row.copy()
  for idx,chunk in enumerate(chunks):
    new_row['chunk-id'] = row['chunk-id'] + f"-{chunks.index(chunk)}"
    new_row['chunk'] = chunk
    dynamic_chunked_arvix.append(new_row)

Dynamically chunking data: 4838it [01:59, 40.52it/s]


In [ ]:
dynamic_chunked_arvix_df = pd.DataFrame(dynamic_chunked_arvix)

In [ ]:
dynamic_chunked_arvix_df.head(2)

In [ ]:
dynamical_chunked_book_df.rename(columns={'text': 'chunk'}, inplace=True)

Reset Index

In [ ]:
dynamical_chunked_book_df = dynamical_chunked_book_df.reset_index(drop=True)
dynamic_chunked_arvix_df = dynamic_chunked_arvix_df.reset_index(drop=True)

In [ ]:
len(documents)

4838

In [ ]:
len(dynamic_chunked_arvix_df)

4869

In [ ]:
len(book_df)

1054

In [ ]:
len(dynamical_chunked_book_df)

1055

## **Vector Database**

In [ ]:
index_name = 'ai-chatbot'

**Create Pinecone Index**

In [ ]:
pc = Pinecone(os.getenv('PINECONE_API'))
index_list = [idx['name'] for idx in pc.list_indexes()]
if index_name not in index_list:
  pc.create_index(name=index_name,spec=ServerlessSpec(cloud='aws',region='us-east-1'),dimension=384)
timeout = 60
start_time = time.time()
while not pc.describe_index(index_name).status['ready']:
  if time.time() - start_time > 60:
    raise TimeoutError("Timeout")
  time.sleep(1)
pc_index = pc.Index(index_name)

In [ ]:
status = pc_index.describe_index_stats()
status.get('total_vector_count',0)

5892

In [ ]:
pc.describe_index(index_name)

In [ ]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

**Sentence Transformer**

In [ ]:
embedding_model = SentenceTransformerEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# pc.delete_index(index_name)

In [ ]:
status = pc_index.describe_index_stats()
status.get('total_vector_count',0)

5892

Upsert pdf data into vector database

In [ ]:
batch_size = 100
if status['total_vector_count'] == 0:
  for i in tqdm(range(0,len(dynamical_chunked_book_df),batch_size)):
    i_end = min(i+batch_size,len(dynamical_chunked_book_df))
    batch = dynamical_chunked_book_df[i:i_end]
    ids = batch['id'].astype(str).to_list()
    chunks = batch['chunk'].to_list()
    embed_chunk = embedding_model.encode(chunks)
    metadata = batch[['id','chunk']].to_dict(orient='records')
    pc_index.upsert(vectors=list(zip(ids,embed_chunk,metadata)))
else:
  print('Already Inserted data')

Already Inserted data


In [ ]:
status = pc_index.describe_index_stats()
status.get('total_vector_count',0)

5892

Upsert arvix dataset into vector database

In [ ]:
status = pc_index.describe_index_stats()
if status.get('total_vector_count', 0) <= 1054:
    for i in tqdm(range(0, len(dynamic_chunked_arvix_df), batch_size)):
        i_end = min(len(dynamic_chunked_arvix_df), i + batch_size)
        batch = dynamic_chunked_arvix_df[i:i_end]
        ids = (batch['doi'].astype(str) + '-' +
               batch['chunk-id'].astype(str)).to_list()
        chunk = batch['chunk'].to_list()
        embeds = embedding_model.encode(chunk)
        meta_data = batch[['chunk', 'source', 'title']
                          ].to_dict(orient='records')
        pc_index.upsert(vectors=list(zip(ids, embeds, meta_data)))
else:
    print("Alredy Created")

Alredy Created


In [ ]:
status = pc_index.describe_index_stats()
status.get('total_vector_count',0)

5892

In [ ]:
query = 'What is Machine Learning?'
vectorstore = PineconeVectorStore(pc_index, embedding_model, text_key='chunk')
contexts = vectorstore.similarity_search(query, k=3)

In [ ]:
contexts

## **Create Model**

In [ ]:
query = 'What is deep learning? How it difer from ML'

In [ ]:
query

'What is deep learning? How it difer from ML'

In [ ]:
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
deepseek_repo = "deepseek-ai/deepseek-llm-67b-base"
deepseek_model =  HuggingFaceEndpoint(repo_id=deepseek_repo,task='text-generation',
    temperature=0.7,  # Increase temperature for less deterministic output
    top_p=0.9,        # Nucleus sampling to encourage diversity
    top_k=50          # Consider the top 50 tokens at each step
)

In [ ]:
repo_id = "meta-llama/Llama-3.3-70B-Instruct"
max_new_tokens = 8192
llama_model = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_new_tokens=max_new_tokens,
    top_k=10,
    top_p=0.95,
    temperature=0.6,
    task='text-generation',
    repetition_penalty=1.03
)

Test Models

In [ ]:
llama_model.invoke(query)

In [ ]:
deepseek_model.invoke(query)

In [ ]:
gemini_model.invoke(query)

In [ ]:
models = [
    gemini_model,
    llama_model,
    deepseek_model
]

## **Auguemnt Prompt**

In [ ]:
from langchain.prompts.chat import (
  ChatPromptTemplate,
  HumanMessagePromptTemplate,
  MessagesPlaceholder,
)
from langchain.schema import SystemMessage

In [ ]:
def augument_prompt(query,no_of_docs=3):
  vectorstore = PineconeVectorStore(pc_index, embedding_model, text_key='chunk')
  contexts = vectorstore.similarity_search(query, k=no_of_docs)

  # from langchain.schema import SystemMessage, HumanMessage, AIMessage, UserMessage

  system_msg = SystemMessage(
      content=(
          "You are a highly knowledgeable AI expert specializing in Artificial Intelligence, "
          "machine learning, and Deep Learning. Answer questions with detailed, clear, and technically accurate explanations. "
          "Always provide examples or analogies when they help clarify complex topics."
      )
  )

  human_msg_template = HumanMessagePromptTemplate.from_template("{user_query}")

  chat_history_placeholder = MessagesPlaceholder(variable_name="chat_history")

  user_msg_template = HumanMessagePromptTemplate.from_template("{additional_context}")

  chat_prompt = ChatPromptTemplate.from_messages([
      system_msg,
      chat_history_placeholder,
      human_msg_template,
      user_msg_template,
  ])

  formatted_prompt = chat_prompt.format(
      user_query=query,
      chat_history=[],
      additional_context = contexts
  )

  return formatted_prompt


In [ ]:
query

In [ ]:
models[0].invoke(augument_prompt(query))

In [ ]:
augument_prompt(query)

In [ ]:
llama_model.invoke(augument_prompt("What is machine AI"))

In [ ]:
models[2].invoke(augument_prompt(query))

## **Test Model**